In [2]:
import os
import os.path as osp

import torch
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

from ogb.nodeproppred import PygNodePropPredDataset


# create directory
folder_path = osp.join(os.getcwd(), 'data')

if not osp.exists(folder_path):
    os.makedirs(folder_path)
    print("Directory '% s' created" % folder_path)


Directory '/home/jharris/Desktop/approx_attention/setup/data' created


In [3]:
path = osp.join(folder_path, 'cora')

if not osp.exists(path):
    os.makedirs(path)

for K in range(6):

    transform = T.Compose([
        T.NormalizeFeatures(),
        T.ToUndirected(),
        T.AddSelfLoops(),
        T.SIGN(K)
    ])

    # download
    dataset = Planetoid(
        root=path,
        name='Cora',
        transform=transform,
        split='full',
    )

    filename = osp.join(path, f'cora_sign_k{K}.pth')
    torch.save(dataset, filename)




Processing...
Done!


In [4]:
path = osp.join(folder_path, 'pubmed')

if not osp.exists(path):
    os.makedirs(path)

for K in range(6):

    transform = T.Compose([
        T.NormalizeFeatures(),
        T.ToUndirected(),
        T.AddSelfLoops(),
        T.SIGN(K)
    ])

    # download
    dataset = Planetoid(
        root=path,
        name='Pubmed',
        transform=transform,
        split='full',
    )

    filename = osp.join(path, f'pubmed_sign_k{K}.pth')
    torch.save(dataset, filename)


Processing...
Done!


In [5]:
path = osp.join(folder_path, 'arxiv')

if not osp.exists(path):
    os.makedirs(path)

for K in range(6):

    transform = T.Compose([
        T.NormalizeFeatures(),
        T.ToUndirected(),
        T.AddSelfLoops(),
        T.SIGN(K)
    ])

    # download
    dataset = PygNodePropPredDataset(
        'ogbn-arxiv',
        root=path,
        transform=transform)

    filename = osp.join(path, f'arxiv_sign_k{K}.pth')
    torch.save(dataset, filename)
    del transform, dataset


Downloaded 0.08 GB: 100%|██████████| 81/81 [00:07<00:00, 11.33it/s]


Extracting /home/jharris/Desktop/approx_attention/setup/data/arxiv/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 14266.34it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 4798.97it/s]

Saving...



Done!


In [6]:
path = osp.join(folder_path, 'products')

if not osp.exists(path):
    os.makedirs(path)

for K in range(6):

    transform = T.Compose([
        T.NormalizeFeatures(),
        T.ToUndirected(),
        T.AddSelfLoops(),
        T.SIGN(K)
    ])

    # download
    dataset = PygNodePropPredDataset(
        'ogbn-products',
        root=path,
        transform=transform)

    filename = osp.join(path, f'products_sign_k{K}.pth')
    torch.save(dataset, filename)
    del transform, dataset


Downloaded 1.38 GB: 100%|██████████| 1414/1414 [07:57<00:00,  2.96it/s]


Extracting /home/jharris/Desktop/approx_attention/setup/data/products/products.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:02<00:00,  2.05s/it]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 4500.33it/s]


Saving...


Done!
